In [1]:
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
from optuna import Trial, visualization



torch.__version__

'1.11.0+cu113'

In [2]:
!nvidia-smi

Thu Apr 21 11:16:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8     4W /  N/A |      0MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
data = pd.read_csv('../Data/train_features_computed_tabular.csv')
labels = pd.read_csv('../Data/train_labels.csv')
data = data.merge(labels, on = 'sequence', how = 'left')
X, y = data.drop(['sequence', 'state'], axis = 1).values, data['state'].values

In [4]:
scaler = StandardScaler()
model=scaler.fit(X)
X=model.transform(X)

In [5]:
def Objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1.0, 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    param = dict(
        n_d=n_da,
        n_a=n_da,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
        mask_type=mask_type,
        n_shared=n_shared,
        scheduler_params=dict(
            mode="min",
            patience=trial.suggest_int("patienceScheduler", low=3, high=10),
            min_lr=1e-5,
            factor=0.5,
        ),
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        verbose=0,
    )
    kf = KFold(n_splits=3, random_state=42, shuffle=True)
    CV_score_array = []
    for train_index, test_index in kf.split(X):
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        clf = TabNetClassifier(**param)
        clf.fit(
            X_train=X_train,
            y_train=y_train,
            eval_set=[(X_valid, y_valid)],
            patience=trial.suggest_int("patience", low=15, high=30),
            max_epochs=trial.suggest_int("epochs", 1, 100),
        )
        preds = clf.predict(X_valid)
        auc = roc_auc_score(y_valid, preds)

        CV_score_array.append(auc)
    avg = np.mean(CV_score_array)
    return avg


In [6]:
study = optuna.create_study(direction="maximize", study_name='TabNet optimization')
study.optimize(Objective, timeout=6*60) #5 hours

best = study.best_params
print('The best parameters are ', best)

[I 2022-04-21 11:16:30,573] A new study created in memory with name: TabNet optimization


Stop training because you reached max_epochs = 8 with best_epoch = 7 and best_val_0_auc = 0.83962
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 8 with best_epoch = 7 and best_val_0_auc = 0.84173
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 8 with best_epoch = 7 and best_val_0_auc = 0.83231
Best weights from best epoch are automatically used!


[I 2022-04-21 11:17:21,571] Trial 0 finished with value: 0.7675103777093201 and parameters: {'mask_type': 'sparsemax', 'n_da': 56, 'n_steps': 2, 'gamma': 1.2, 'n_shared': 3, 'lambda_sparse': 9.271707091323517e-06, 'patienceScheduler': 9, 'patience': 17, 'epochs': 8}. Best is trial 0 with value: 0.7675103777093201.



Early stopping occurred at epoch 30 with best_epoch = 14 and best_val_0_auc = 0.86183
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 30 with best_epoch = 14 and best_val_0_auc = 0.87259
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 35 with best_epoch = 19 and best_val_0_auc = 0.86842
Best weights from best epoch are automatically used!


[I 2022-04-21 11:19:20,644] Trial 1 finished with value: 0.7924097928755233 and parameters: {'mask_type': 'entmax', 'n_da': 60, 'n_steps': 1, 'gamma': 1.0, 'n_shared': 2, 'lambda_sparse': 0.0002052578825220565, 'patienceScheduler': 5, 'patience': 16, 'epochs': 99}. Best is trial 1 with value: 0.7924097928755233.


Stop training because you reached max_epochs = 37 with best_epoch = 13 and best_val_0_auc = 0.86441
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 35 with best_epoch = 11 and best_val_0_auc = 0.87056
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 36 with best_epoch = 12 and best_val_0_auc = 0.86137
Best weights from best epoch are automatically used!


[I 2022-04-21 11:21:21,278] Trial 2 finished with value: 0.7912998177671828 and parameters: {'mask_type': 'entmax', 'n_da': 64, 'n_steps': 1, 'gamma': 1.2, 'n_shared': 1, 'lambda_sparse': 0.0007776235991341509, 'patienceScheduler': 9, 'patience': 24, 'epochs': 37}. Best is trial 1 with value: 0.7924097928755233.


Stop training because you reached max_epochs = 40 with best_epoch = 23 and best_val_0_auc = 0.86637
Best weights from best epoch are automatically used!
Stop training because you reached max_epochs = 40 with best_epoch = 21 and best_val_0_auc = 0.86728
Best weights from best epoch are automatically used!

Early stopping occurred at epoch 35 with best_epoch = 15 and best_val_0_auc = 0.85917
Best weights from best epoch are automatically used!


[I 2022-04-21 11:24:00,037] Trial 3 finished with value: 0.7897864020056652 and parameters: {'mask_type': 'sparsemax', 'n_da': 60, 'n_steps': 1, 'gamma': 1.0, 'n_shared': 2, 'lambda_sparse': 0.00035600368131822626, 'patienceScheduler': 6, 'patience': 20, 'epochs': 40}. Best is trial 1 with value: 0.7924097928755233.


The best parameters are  {'mask_type': 'entmax', 'n_da': 60, 'n_steps': 1, 'gamma': 1.0, 'n_shared': 2, 'lambda_sparse': 0.0002052578825220565, 'patienceScheduler': 5, 'patience': 16, 'epochs': 99}


In [7]:
best_model_params = dict(
    n_d=best["n_da"],
    n_a=best["n_da"],
    n_steps=best["n_steps"],
    gamma=best["gamma"],
    lambda_sparse=best["lambda_sparse"],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
    mask_type=best["mask_type"],
    n_shared=best["n_shared"],
    scheduler_params=dict(
        mode="min",
        patience=best["patienceScheduler"],
        min_lr=1e-5,
        factor=0.5,
    ),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    verbose=0,
)
epochs = best["epochs"]

clf = TabNetClassifier(**best_model_params)
clf.fit(X_train=X, y_train=y,
          patience=best['patience'], max_epochs=epochs,
          eval_metric=['auc'])


No early stopping will be performed, last training weights will be used.


In [16]:
test = pd.read_csv('../Data/test_features_computed_tabular.csv')

In [17]:
test.drop('sequence', axis = 1, inplace = True)
test = model.transform(np.array(test))

In [19]:
preds = clf.predict(test)

In [25]:
sub = pd.read_csv('../Data/sample_submission.csv')

In [26]:
sub['state'] = preds
sub.to_csv('../Submissions/tabnet_sub.csv', index = False)

In [27]:
sub.state.value_counts()

1    6352
0    5866
Name: state, dtype: int64

In [28]:
sub

,sequence,state
0,25968,0
1,25969,1
2,25970,0
3,25971,0
4,25972,1
...,...,...
12213,38181,0
12214,38182,0
12215,38183,1
12216,38184,0
